In [4]:
import numpy as np
import trimesh
from sklearn.decomposition import PCA
import os

# Step 3.1: Full Normalization

In [5]:
def normalize_shape(shape: trimesh.Trimesh):
    # Centering and Scaling (from Step 2)
    shape.vertices -= shape.center_mass
    shape.vertices /= np.linalg.norm(shape.bounding_box_oriented.extents)

    # Alignment using PCA
    pca = PCA(n_components=3)
    pca.fit(shape.vertices)
    aligned_vertices = pca.transform(shape.vertices)
    shape.vertices = aligned_vertices

    # Flipping using moment test
    moments = np.mean(shape.vertices, axis=0)
    for axis in range(3):
        if moments[axis] < 0:
            shape.vertices[:, axis] *= -1

    return shape

# Step 3.2: Feature Extraction

In [6]:
def extract_features(shape: trimesh.Trimesh):
    features = {}
    
    # Elementary Descriptors
    features['surface_area'] = shape.area
    features['volume'] = shape.volume
    features['compactness'] = (shape.area ** 3) / (36 * np.pi * (shape.volume ** 2))
    obb = shape.bounding_box_oriented
    features['rectangularity'] = shape.volume / obb.volume
    features['diameter'] = np.linalg.norm(shape.bounding_box_oriented.extents)
    convex_hull = shape.convex_hull
    features['convexity'] = shape.volume / convex_hull.volume
    covariance_matrix = np.cov(shape.vertices, rowvar=False)
    eigenvalues, _ = np.linalg.eig(covariance_matrix)
    features['eccentricity'] = max(eigenvalues) / min(eigenvalues)

    # Property Descriptors (using histograms)
    def compute_histogram(values, bins=10):
        hist, _ = np.histogram(values, bins=bins, density=True)
        return hist
    
    random_vertices = shape.vertices[np.random.choice(shape.vertices.shape[0], 1000, replace=True)]
    features['A3'] = compute_histogram([np.arccos(np.clip(np.dot(random_vertices[i], random_vertices[j]) / \
                                                   (np.linalg.norm(random_vertices[i]) * np.linalg.norm(random_vertices[j])), -1.0, 1.0))
                                        for i in range(len(random_vertices)) for j in range(i+1, len(random_vertices))], bins=10)
    
    barycenter = shape.center_mass
    features['D1'] = compute_histogram([np.linalg.norm(vertex - barycenter) for vertex in random_vertices], bins=10)
    features['D2'] = compute_histogram([np.linalg.norm(random_vertices[i] - random_vertices[j])
                                        for i in range(len(random_vertices)) for j in range(i+1, len(random_vertices))], bins=10)
    features['D3'] = compute_histogram([np.sqrt(trimesh.triangles.area(random_vertices[[i, j, k]]))
                                        for i in range(len(random_vertices)) for j in range(i+1, len(random_vertices))
                                        for k in range(j+1, len(random_vertices))], bins=10)
    features['D4'] = compute_histogram([np.cbrt(np.abs(np.dot(random_vertices[i], np.cross(random_vertices[j], random_vertices[k]))))
                                        for i in range(len(random_vertices)) for j in range(i+1, len(random_vertices))
                                        for k in range(j+1, len(random_vertices))], bins=10)
    
    return features

In [ ]:
# Example usage

def load_and_extract_features_from_directory(directory_path):
    for filename in os.listdir(directory_path):
        if filename.endswith('.obj'):
            shape_path = os.path.join(directory_path, filename)
            try:
                shape = trimesh.load(shape_path)
                # Step 3.1: Normalize the shape
                normalized_shape = normalize_shape(shape)
                # Step 3.2: Extract features
                features = extract_features(normalized_shape)
                # Print extracted features
                print(f"Features for {filename}: {features}")
            except Exception as e:
                print(f"Error loading or processing shape {filename}: {e}")

# Specify the path to your directory containing shape files
directory_path = "path_to_your_directory"
load_and_extract_features_from_directory(directory_path)